In [1]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv

import os

In [2]:
load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')

In [3]:
upload_resume_prompt = """Analyze the provided document and retrieve the following data:
- Name
- Experience (in years) (Calculate based on experience if not mentioned) (Need just give a number nothing else, no reasoning)
- Domain (e.g., Data Science, Fullstack Development) (Need to strictly be specified to IT industry)
- Skills (e.g., Python, Java, Azure)

Here's an example format for returning the data:

name: John Doe
experience: 5 (You need to strictly give just a number, nothing else)
domain: Fullstack Development
skills: Python, Java, Azure, MongoDB

Document to analyze: {data}

Note: Follow the example format strictly, including letter casing. Omit Employee ID if not mentioned.
"""

In [4]:
loader = UnstructuredFileLoader(r"C:\Users\PATALLAP\Downloads\46328674 - Panindhra Tallapudi.pdf")
load = loader.load()
load

c:\FastAPI Projects\HRTech.API\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(page_content='Panindhra Tallapudi Analyst/Software Engineer\n\nExperience : 2 years\n\nBase Location:\n\nBangalore\n\nEmail ID:\n\npanindhra.a.tallapudi@gmail.com\n\nMobile No:\n\n+91 8919512912\n\nGrade:\n\nA4\n\nStrengths\n\nAchievement\n\nPractical exposure to creating microservices with .NET Web API and implementing JWT authentication and database integration\n\nTrain Reservation System\n\n\n\nIntermediate-level proficiency in developing microservices with .NET Framework and .NET Web API and FastAPI using Python\n\nExperience in implementing swagger for testing the\n\nAccomplished an end-to-end case study of a Train Reservation System, incorporating JWT authentication, Swagger for API documentation, payment integration, and utilizing Angular for the user interface.\n\nApplication Programming Interface.\n\n\n\nIn-depth understanding of deploying .NET applications.\n\nPractical experience in Angular client side application development which is integrated with the API of the

In [5]:

from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class EmployeeInformation(BaseModel):
    """Information about an image."""
    employee_id: str = Field(description="Employee id in the document")
    name: str = Field(description="Name of the employee in the document")
    experience: str = Field(description="Experience of the employee in the document")
    domain: str = Field(description="Domain in the document, if not mentioned try to figure it out with the details in the document")
    skills: List[str] = Field(description="List of skills mentioned by employee in the document")

In [6]:
from langchain_core.output_parsers import JsonOutputParser

In [7]:
parser = JsonOutputParser(pydantic_object=EmployeeInformation)
parser.get_format_instructions()


'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Information about an image.", "properties": {"employee_id": {"title": "Employee Id", "description": "Employee id in the document", "type": "string"}, "name": {"title": "Name", "description": "Name of the employee in the document", "type": "string"}, "experience": {"title": "Experience", "description": "Experience of the employee in the document", "type": "string"}, "domain": {"title": "Domain", "description": "Domain in the document, if not mentioned try to figure it out with the details in the document", "type": "stri

In [8]:
prompt = PromptTemplate(
                input_variables=["data"], template=upload_resume_prompt, partial_variables={"format_instructions": parser.get_format_instructions()}
            )

def get_llm(temperature=0.5):
        return AzureChatOpenAI(
            openai_api_version="2023-12-01-preview",
            azure_deployment="<your_deployment_name>",
            max_tokens=5000,
            temperature = temperature
        )


In [9]:
llm_chain = LLMChain(
    llm = get_llm(),
    prompt = prompt
)

In [10]:
data = llm_chain(load)

c:\FastAPI Projects\HRTech.API\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
data["text"]

'name: Panindhra Tallapudi\nexperience: 2\ndomain: IT industry\nskills: C#, Python, .NET Core Framework, ASP .NET MVC, ASP .NET Web API, JWT Authentication, Python, FastAPI, Langchain Framework, Web Angular, Typescript, Database Microsoft SQL Server, Vector database, Web Technologies HTML5, CSS3, Javascript, Bootstrap, Add Ons Swagger, Git-hub, Docker'